In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import os

In [ ]:
base_url = 'https://www.fungipedia.org/hongos.html?start='
driver = webdriver.Chrome()
for i in range(0, 29): # Hay 29 paginas
    print(f'Starting page {i+1}...', end='\r')
    driver.get(base_url + str(20*i))
    fungi = driver.find_elements(By.CLASS_NAME, 'catItemViewFungi')
    urls = [bird.find_element(By.TAG_NAME, 'a').get_attribute('href') for bird in fungi]
    scientific_names = []
    descriptions = []
    for url in urls:
        driver.get(url)
        scientific_name = driver.find_element(By.CLASS_NAME, 'itemTitle').text
        try:
            description = driver.find_element(By.CLASS_NAME, 'itemFullText').text
        except:
            print(f'Error finding description of {scientific_name}')
            continue
        description = description.split('Hábitat:')[0]
        description = description.replace('Características:', '')
        description = description.replace('\n', ' ').strip()
        scientific_names.append(scientific_name)
        descriptions.append(description)
    df = pd.DataFrame({'Nombre aceptado': scientific_names, 'Descripción': descriptions})
    df['Grupo taxonómico'] = 'Hongos'
    df['Fuente'] = 'Fungipedia'
    os.makedirs('Datos Fungipedia', exist_ok=True)
    df.to_excel('Datos Fungipedia/fungi_descriptions_Fungipedia.xlsx', index=False)
driver.close()